In [ ]:
import pandas as pd
import json

# Entities

In [ ]:
wikia = pd.read_csv("info/entities.wikia.csv", 
                    names=["id", "name", "url"],
                    usecols=["name", "url"], 
                    header=0, index_col=["url"])
gamepedia = pd.read_csv("info/entities.gamepedia.csv", 
                        names=["id", "name", "url"],
                        usecols=["name", "url"], 
                        header=0, index_col=["url"])

In [ ]:
mixed = pd.concat([wikia, gamepedia])
print(mixed.info())

unique_names = pd.DataFrame({'name': mixed.name.unique()})
unique_urls = pd.DataFrame({'url':mixed.index.unique()})

unique_names.to_csv("info/entities.csv")
unique_urls.to_csv("info/urls.csv")

print("Unique names", len(unique_names))
print("Unique urls", len(unique_urls))

mixed.sample(5)

### Insert in neo4j

```
LOAD CSV WITH HEADERS FROM "file:///entities.csv" AS line WITH line
CREATE (:Entity{name:line.name})
```

```
CREATE INDEX ON :Entity(name)
```

```
LOAD CSV WITH HEADERS FROM "file:///urls.csv" AS line WITH line
CREATE (:Page{url:line.url})
```

```
CREATE INDEX ON :Page(url)
```

```
LOAD CSV WITH HEADERS FROM "file:///entities.wikia.csv" AS line WITH line
MATCH (e:Entity{name:line.name})
MATCH (p:Page{url:line.page})
MERGE (e)-[:Website{name:"wikia"}]->(p)
```

```
LOAD CSV WITH HEADERS FROM "file:///entities.gamepedia.csv" AS line WITH line
MATCH (e:Entity{name:line.name})
MATCH (p:Page{url:line.page})
MERGE (e)-[:Website{name:"gamepedia"}]->(p)
```

## Merge on URL

In [ ]:
joint = pd.merge(wikia, gamepedia, left_index=True, right_index=True,suffixes=('_wikia', '_gamepedia'), how='outer')
joint.head()

In [ ]:
shared_entities = joint.loc[pd.notnull(joint["name_wikia"]) & pd.notnull(joint["name_gamepedia"])]
only_wikia =joint.loc[pd.notnull(joint["name_wikia"]) & pd.isnull(joint["name_gamepedia"])]
only_gamepedia = joint.loc[pd.isnull(joint["name_wikia"]) & pd.notnull(joint["name_gamepedia"])]

In [ ]:
print(shared_entities.info())
shared_entities.sample(5)

In [ ]:
print("Duplicated", len(shared_entities[shared_entities.index.duplicated()]))

## Merge on name

In [ ]:
joint_names = pd.merge(wikia.reset_index(), 
                       gamepedia.reset_index(), 
                       left_on="name", right_on="name",
                       suffixes=('_wikia', '_gamepedia'), 
                       how='outer').set_index("name")

shared_entities_name = joint_names.loc[pd.notnull(joint_names["url_wikia"]) & 
                                  pd.notnull(joint_names["url_gamepedia"])&
                                             (joint_names["url_gamepedia"]!= joint_names["url_wikia"])]
print(shared_entities_name.info())
shared_entities_name.sample(5)

## Extract connections

In [ ]:
url_wikia_joint = joint_names.groupby(joint_names.index)["url_wikia"].apply(set)
url_gamepedia_joint = joint_names.groupby(joint_names.index)["url_gamepedia"].apply(set)

data_urls = list(zip(url_wikia_joint.index.values, url_wikia_joint.values, url_gamepedia_joint.values))

In [ ]:
connections = []
for d in data_urls:
    name = d[0]
    for wikia_url in d[1]:
        if pd.notna(wikia_url):
            connections.append([name, 'wikia', wikia_url])
    for gamepedia_url in d[2]:
        if pd.notna(gamepedia_url):
            connections.append([name, 'gamepedia', gamepedia_url])

connections_urls = pd.DataFrame(connections, columns=['name', 'site', 'url'])
print(connections_urls.info())
connections_urls.sample(5)

In [ ]:
name_wikia_joint = joint.groupby(joint.index)["name_wikia"].apply(set)
name_gamepedia_joint = joint.groupby(joint.index)["name_gamepedia"].apply(set)
data_names = list(zip(name_wikia_joint.index.values, name_wikia_joint.values, name_wikia_joint.values))

In [ ]:
connections = []
for d in data_names:
    url = d[0]
    for wikia_name in d[1]:
        if pd.notna(wikia_name):
            connections.append([wikia_name, 'wikia', url])
    for gamepedia_name in d[2]:
        if pd.notna(gamepedia_name):
            connections.append([gamepedia_name, 'gamepedia', url])

connections_names = pd.DataFrame(connections, columns=['name', 'site', 'url'])
print(connections_names.info())
connections_names.sample(5)

In [ ]:
connections_final = pd.concat([connections_urls, connections_names])
connections_final.info()
connections_final.sample(5)

In [ ]:
unique_names = pd.DataFrame({'name': connections_final.name.unique()})
unique_urls = pd.DataFrame({'url':connections_final.url.unique()})

print("Unique names", len(unique_names))
print("Unique urls", len(unique_urls))

(I just realized that I already had this...)